#### Для месторождения Капмо Трес необходимо cформировать финансово - экономическую модель и рассчитать показатели разработки:
* Ввести данные по профилю добычи
* Ввести данные по вводу скважины
* Ввести данные по бурению/кустованию 
* Ввести данные по схеме поверхностного обустройства 
* Рассчитать финансово-экономические показатели 

In [1]:
import plotly.graph_objs as go
import pandas as pd
from plotly.offline import init_notebook_mode

init_notebook_mode(connected = True)

*1) Вводим статические параметры пласта и обустройства месторождения:*

In [2]:
depth = 2500 # глубина пласта, м
pads_number = 4 # количество кустов
horizontal_length = 700 # длина горизонтального участка ствола добывающих ГС, м
HF_halflength = 500 # полудлина трещины ГРП (скин=-2), м


*2) Вводим сценарные экономические условия (задаются преподавателем); их можно будет менять для анализа чувствительности:*

In [3]:
oil_price = 15000 # цена нефти в руб/т
oil_taxes = 10500 # налоги в руб/т нефти
transport_price = 500 # стоимость транспорта нефти в руб/т
discount_rate = 15 # ставка дисконтирования

*3) Загружаем профили добычи жидкости, нефти, воды и газа из симулятора:*

In [5]:
df = pd.read_csv('csv_folder/MODEL.0.csv', parse_dates=[0], index_col=[0])
j, y, ind_j = 0, 0, 0
time_list, q_oil, q_gas, q_wat, inj_wat = [], [], [], [], []
to = df.index[0]
fopt, fgpt, fwpt, fwit, flpt = [], [], [], [], []
for t in df.index:
    if abs((t - to).days) >= 360:
        time_list.append(2020+y)
        to = t
        if y == 0:
            fopt.append(round(df['FOPT'][j]/1000,2)) # добыча нефти по годам, тыс. т
            fgpt.append(round(df['FGPT'][j]/1000000,2)) # добыча газа по годам, млн м3
            fwpt.append(round(df['FWPT'][j]/1000,2)) # добыча воды по годам, тыс т
            fwit.append(round(df['FWIT'][j]/1000,2)) # закачка воды по годам, тыс м3
        else:
            fopt.append(round(df['FOPT'][j]/1000-df['FOPT'][ind_j]/1000, 2)) # добыча нефти по годам, тыс. т
            fgpt.append(round(df['FGPT'][j]/1000000-df['FGPT'][ind_j]/1000000,2)) # добыча газа по годам, млн м3
            fwpt.append(round(df['FWPT'][j]/1000-df['FWPT'][ind_j]/1000,2)) # добыча воды по годам, тыс т
            fwit.append(round(df['FWIT'][j]/1000-df['FWIT'][ind_j]/1000,2)) # закачка воды по годам, тыс м3
        ind_j = j
        y += 1
    j += 1
flpt = [round(x + y,2) for x, y in zip(fopt, fwpt)] # добыча жидкости = добыча нефти + добыча воды
prod_fig = go.Figure()
prod_fig.add_trace(go.Scatter(x=time_list, y=fopt, name='Добыча нефти, тыс. т'))
prod_fig.add_trace(go.Scatter(x=time_list, y=flpt, name='Добыча жидкости, тыс. т'))
prod_fig.add_trace(go.Scatter(x=time_list, y=fgpt, name='Добыча попутного газа, млн нм3'))
prod_fig.add_trace(go.Scatter(x=time_list, y=fwit, name='Закачка воды, тыс. м3'))
prod_fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  title="Основные показатели разработки")
prod_fig.show()

*4) Вводим экономические показатели бурения скважин и данные о их вводе в эксплуатацию:*

In [6]:
drilling_speeds = [225, 225, 270, 270] # скорости проходки БУ1, БУ2, и т.д.; определенные значения для разных типов БУ (180, 225, 270)
MD_drilling_prices, HW_prices, HF_prices, intensification_prices = [], [], [], []
for speed in drilling_speeds:
    MD_drilling_price = speed/100*100000 # стоимость проходки по стволу, руб/м
    HW_price = speed/100*160000 # стоимость бурения ГС, руб/м
    HF_price = speed/100*200000 # стоимость ГРП, руб/м полудлины трещины
    intensification_price = speed/100*2000000 # стоимость ОПЗ, руб/м полудлины трещины
    MD_drilling_prices.append(MD_drilling_price)
    HW_prices.append(HW_price)
    HF_prices.append(HF_price)
    intensification_prices.append(intensification_price)
i = len(time_list)
prod_direct_wells = [[2, 1], [2, 0], [2, 0], [1, 0]] # ввод добывающих ННС без ГРП; основной список содержит вложенные списки - кусты, которые содержат элементы - кол-во скважин, введенных в экспл. в i-м году
for k in range(0, len(prod_direct_wells)):
    for j in range(len(prod_direct_wells[k]), i):
        prod_direct_wells[k].append(0)
        j+=1
prod_direct_wells_with_HF = [[0, 0], [0, 0], [0, 0], [0, 0]] # ввод добывающих ННС с ГРП
for k in range(0, len(prod_direct_wells_with_HF)):
    for j in range(len(prod_direct_wells_with_HF[k]), i):
        prod_direct_wells_with_HF[k].append(0)
        j+=1
prod_direct_wells_with_intense = [[0, 0], [0, 0], [0, 0], [0, 0]] # ввод добывающих ННС с ОПЗ
for k in range(0, len(prod_direct_wells_with_intense)):
    for j in range(len(prod_direct_wells_with_intense[k]), i):
        prod_direct_wells_with_intense[k].append(0)
        j+=1
prod_horizontal_wells = [[], [], [], []] # ввод добывающих ГС
for k in range(0, len(prod_horizontal_wells)):
    for j in range(len(prod_horizontal_wells[k]), i):
        prod_horizontal_wells[k].append(0)
        j+=1
inj_direct_wells = [[0, 1], [0, 1], [0, 2], [1, 1]] # ввод нагнетательных ННС
for k in range(0, len(inj_direct_wells)):
    for j in range(len(inj_direct_wells[k]), i):
        inj_direct_wells[k].append(0)
        j+=1

*5) Рассчитаем доходы от продажи нефти:*

In [7]:
incomes = []
for oil_rate in fopt:
    sales = oil_rate * oil_price * 1000 # доход от продажи, руб
    taxes = oil_rate * oil_taxes * 1000 # налог, руб
    transport_outlay = oil_rate * transport_price * 1000 # расходы на транспорт нефти, руб
    income = sales - taxes - transport_outlay # чистый доход, руб
    incomes.append(income)

*6) Рассчитаем капитальные затраты на бурение скважин:*

In [9]:
drilling_CAPEX_list = []
j = 0
for time in time_list: # кап.затраты по годам
    drilling_CAPEX = 0
    for i in range(0, pads_number): # кап.затраты на данный год складываются по всем кустам
        prod_direct_wells_CAPEX = depth * MD_drilling_prices[i] * prod_direct_wells[i][j] # кап.затраты на строительство добывающих ННС без ГРП
        prod_direct_wells_with_HF_CAPEX = (depth * MD_drilling_prices[i] + HF_halflength *  HF_prices[i]) * prod_direct_wells_with_HF[i][j] # кап.затраты на строительство добывающих ННС с ГРП
        prod_direct_wells_with_intense_CAPEX = (depth * MD_drilling_prices[i] + intensification_prices[i]) * prod_direct_wells_with_intense[i][j] # кап.затраты на строительство добывающих ННС с ОПЗ
        prod_horizontal_wells_CAPEX = (depth * MD_drilling_prices[i] + horizontal_length * HW_prices[i]) * prod_horizontal_wells[i][j] # кап.затраты на строительство добывающих ГС
        inj_direct_wells_CAPEX = depth * MD_drilling_prices[i] * inj_direct_wells[i][j] # кап.затраты на строительство нагнетательных ННС
        drilling_CAPEX += (prod_direct_wells_CAPEX + prod_direct_wells_with_HF_CAPEX + prod_direct_wells_with_intense_CAPEX + prod_horizontal_wells_CAPEX + inj_direct_wells_CAPEX) # кап.затраты на строительство нагнетательных ННС
    drilling_CAPEX_list.append(drilling_CAPEX)
    j+=1
print(drilling_CAPEX_list)

[4950000000.0, 3712500000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


*7) Рассчитаем капитальные затраты на обустройство месторождения:*